# Module

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm

# Dataset

In [2]:
train_covid_path = '/content/drive/MyDrive/Coding_study/data/train/Covid/'
train_normal_path = '/content/drive/MyDrive/Coding_study/data/train/Normal/'
test_covid_path = '/content/drive/MyDrive/Coding_study/data/test/Covid/'
test_normal_path = '/content/drive/MyDrive/Coding_study/data/test/Normal/'

In [3]:
train_covid = glob(train_covid_path + '*')
train_normal = glob(train_normal_path + '*')
test_covid = glob(test_covid_path + '*')
test_normal = glob(test_covid_path + '*')

In [4]:
print('Lengh of train covid : ',len(train_covid))
print('Lengh of train normal : ',len(train_normal))
print('Lengh of test covid : ',len(test_covid))
print('Lengh of test normal : ',len(test_normal))

Lengh of train covid :  111
Lengh of train normal :  70
Lengh of test covid :  26
Lengh of test normal :  26


In [5]:
X_train = []
X_test = []

In [6]:
for t in tqdm(train_covid):
  img = Image.open(t).convert('RGB').resize((224,224))
  img_arr = np.array(img)
  X_train.append(img_arr)

for t in tqdm(train_normal):
  img = Image.open(t).convert('RGB').resize((224,224))
  img_arr = np.array(img)
  X_train.append(img_arr)

100%|██████████| 70/70 [00:49<00:00,  1.42it/s]


In [7]:
for t in tqdm(test_covid):
  img = Image.open(t).convert('RGB').resize((224,224))
  img_arr = np.array(img)
  X_test.append(img_arr)

for t in tqdm(test_normal):
  img = Image.open(t).convert('RGB').resize((224,224))
  img_arr = np.array(img)
  X_test.append(img_arr)

100%|██████████| 26/26 [00:00<00:00, 66.23it/s]


In [8]:
y_train = [1] * len(train_covid) + [0] * len(train_normal)
y_test = [1] * len(test_covid) + [0] * len(test_normal)

In [9]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [43]:
import random
indicies = list(range(len(X_train)))
random.shuffle(indicies)
print(indicies)

[22, 94, 178, 88, 75, 82, 89, 60, 35, 177, 90, 6, 68, 52, 158, 59, 8, 133, 79, 124, 100, 131, 32, 76, 174, 80, 142, 31, 162, 107, 115, 175, 62, 7, 137, 58, 28, 96, 101, 42, 2, 156, 102, 85, 43, 71, 114, 10, 134, 38, 64, 56, 108, 70, 170, 99, 112, 91, 144, 180, 168, 41, 84, 135, 15, 116, 78, 152, 179, 119, 140, 3, 46, 57, 45, 149, 129, 153, 165, 55, 51, 110, 120, 147, 77, 159, 20, 97, 48, 9, 136, 39, 103, 23, 171, 104, 49, 118, 17, 106, 122, 67, 128, 98, 95, 173, 109, 151, 167, 169, 0, 16, 154, 130, 13, 123, 36, 139, 127, 145, 87, 86, 54, 143, 24, 1, 113, 37, 74, 25, 138, 160, 117, 5, 148, 12, 18, 26, 105, 111, 21, 65, 63, 72, 126, 176, 81, 40, 172, 83, 29, 155, 14, 61, 93, 125, 19, 150, 66, 50, 146, 47, 27, 132, 30, 163, 69, 33, 121, 34, 11, 53, 166, 4, 73, 164, 141, 157, 44, 92, 161]


In [44]:
# shuffle
X_train = X_train[indicies]
y_train = y_train[indicies]

In [45]:
print('Shape of X train : ',X_train.shape)
print('Shape of y train : ',y_train.shape)
print('Shape of X test : ',X_test.shape)
print('Shape of y test : ',y_test.shape)

Shape of X train :  (181, 224, 224, 3)
Shape of y train :  (181,)
Shape of X test :  (52, 224, 224, 3)
Shape of y test :  (52,)


# Feature based

사전 학습의 결과값을 하나의 입력으로 활용하는 방법

### VGG16

In [11]:
vgg = tf.keras.applications.VGG16(include_top=True,input_shape=(224,224,3))

553467096/553467096 [==============================] - 25s 0us/step


사전 학습이 vgg16은 224,224 크기로 고정되어 있음

In [12]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
vgg(X_train[:1,:,:,:]).shape

TensorShape([1, 1000])

사전 학습이 1000 개의 class를 구별하도록 학습되었기 때문에 출력층의 hidden node 개수가 1000개

우리는 2 가지를 분류하고 싶음 -> Output node의 수가 2여야 함

In [14]:
vgg = tf.keras.applications.VGG16(include_top=False,input_shape=(224,224,3))

58889256/58889256 [==============================] - 4s 0us/step


In [15]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

include_top은 전결합 계층을 포함할 것인지에 대해 인자

해당 인자가 거짓일 경우에는 전결합 계층 전까지만 사전 학습된 모형을 불러와줌

In [16]:
from sklearn.linear_model import LogisticRegression

In [28]:
X_train_vgg = vgg.predict(X_train,batch_size=32)

6/6 [==============================] - 8s 699ms/step


In [29]:
X_train_vgg.shape

(181, 7, 7, 512)

In [30]:
X_train_vgg = tf.reshape(X_train_vgg,(181,-1))
X_train_vgg.shape

TensorShape([181, 25088])

In [31]:
lr = LogisticRegression()

In [32]:
lr.fit(X_train_vgg,y_train)

LogisticRegression()

In [33]:
X_test_vgg = vgg(X_test)
X_test_vgg = tf.reshape(X_test_vgg,(-1,25088))
X_test_vgg.shape

TensorShape([52, 25088])

In [34]:
lr.score(X_test_vgg,y_test)

0.5

# 미세 조정(Fine-tuning)

마지막 출력층을 조정하여서 전체적으로 다시 학습을 하는 방법.

다만, 사전 학습된 모형의 가중치를 고정시킬 수 있음

In [36]:
# 사전 학습 가중치 고정
inputs = tf.keras.layers.Input(shape=(224,224,3))
vgg = tf.keras.applications.VGG16(input_shape=(224,224,3),include_top=False)
vgg.trainable=False # 가중치 고정
x = vgg(inputs)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(2,activation='softmax')(x)
model = tf.keras.models.Model(inputs,outputs)

In [37]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense_1 (Dense)             (None, 2)                 50178     
                                                                 
Total params: 14,764,866
Trainable params: 50,178
Non-trainable params: 14,714,688
_________________________________________________________________


In [38]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [46]:
model.fit(X_train,y_train,epochs=5,batch_size=32)

Epoch 1/5
6/6 [==============================] - 1s 116ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/5
6/6 [==============================] - 1s 110ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/5
6/6 [==============================] - 1s 112ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/5
6/6 [==============================] - 1s 110ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/5
6/6 [==============================] - 1s 111ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [47]:
model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 101ms/step - loss: 78.2687 - accuracy: 0.5000


[78.2686767578125, 0.5]

In [48]:
# 사전 학습 가중치 고정 X
inputs = tf.keras.layers.Input(shape=(224,224,3))
vgg = tf.keras.applications.VGG16(input_shape=(224,224,3),include_top=False)
x = vgg(inputs)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(2,activation='softmax')(x)
model = tf.keras.models.Model(inputs,outputs)

In [49]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 2)                 50178     
                                                                 
Total params: 14,764,866
Trainable params: 14,764,866
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [51]:
model.fit(X_train,y_train,epochs=5,batch_size=32)

Epoch 1/5
6/6 [==============================] - 19s 1s/step - loss: 48.0595 - accuracy: 0.5304
Epoch 2/5
6/6 [==============================] - 2s 346ms/step - loss: 3.0200 - accuracy: 0.4751
Epoch 3/5
6/6 [==============================] - 2s 344ms/step - loss: 0.7552 - accuracy: 0.5967
Epoch 4/5
6/6 [==============================] - 2s 343ms/step - loss: 0.7911 - accuracy: 0.6630
Epoch 5/5
6/6 [==============================] - 2s 343ms/step - loss: 0.6578 - accuracy: 0.6519


In [52]:
model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 92ms/step - loss: 0.7624 - accuracy: 0.5000


[0.7624354958534241, 0.5]

이외에 모형도 위와 같이 불러와서 사용가능

불러올 수 있는 모형은 아래 참고

[https://www.tensorflow.org/api_docs/python/tf/keras/applications](https://www.tensorflow.org/api_docs/python/tf/keras/applications)

In [59]:
resnet = tf.keras.applications.ResNet50()

102967424/102967424 [==============================] - 6s 0us/step


In [60]:
help(tf.keras.applications)

Help on package keras.api._v2.keras.applications in keras.api._v2.keras:

NAME
    keras.api._v2.keras.applications - Keras Applications are premade architectures with pre-trained weights.

PACKAGE CONTENTS
    convnext (package)
    densenet (package)
    efficientnet (package)
    efficientnet_v2 (package)
    imagenet_utils (package)
    inception_resnet_v2 (package)
    inception_v3 (package)
    mobilenet (package)
    mobilenet_v2 (package)
    mobilenet_v3 (package)
    nasnet (package)
    regnet (package)
    resnet (package)
    resnet50 (package)
    resnet_rs (package)
    resnet_v2 (package)
    vgg16 (package)
    vgg19 (package)
    xception (package)

FILE
    /usr/local/lib/python3.10/dist-packages/keras/api/_v2/keras/applications/__init__.py




# ViT fine-tuning

In [62]:
!pip install --quiet vit-keras

In [66]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 8.9 MB/s eta 0:00:00


In [67]:
from vit_keras import vit

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [80]:
vit_model = vit.vit_b32(
        image_size = (224,224),
        pretrained = True,
        include_top = False,
        pretrained_top = False,)

/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 7, 7
  warnings.warn(


In [70]:
vit_model.summary()

Model: "vit-b32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 7, 7, 768)         2360064   
                                                                 
 reshape (Reshape)           (None, 49, 768)           0         
                                                                 
 class_token (ClassToken)    (None, 50, 768)           768       
                                                                 
 Transformer/posembed_input   (None, 50, 768)          38400     
 (AddPositionEmbs)                                               
                                                                 
 Transformer/encoderblock_0   ((None, 50, 768),        7087872   
 (TransformerBlock)           (None, 12, None, None))      

In [74]:
inputs = tf.keras.layers.Input(shape=(224,224,3))
x = vit_model(inputs)
outputs = tf.keras.layers.Dense(2,activation='softmax')(x)
model = tf.keras.models.Model(inputs,outputs)

In [75]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 vit-b32 (Functional)        (None, 768)               87455232  
                                                                 
 dense_5 (Dense)             (None, 2)                 1538      
                                                                 
Total params: 87,456,770
Trainable params: 87,456,770
Non-trainable params: 0
_________________________________________________________________


In [76]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [77]:
model.fit(X_train,y_train,epochs=5,batch_size=32)

Epoch 1/5
6/6 [==============================] - 50s 320ms/step - loss: 2.4805 - accuracy: 0.5138
Epoch 2/5
6/6 [==============================] - 2s 288ms/step - loss: 0.7888 - accuracy: 0.5580
Epoch 3/5
6/6 [==============================] - 2s 289ms/step - loss: 0.7747 - accuracy: 0.4530
Epoch 4/5
6/6 [==============================] - 2s 289ms/step - loss: 0.6971 - accuracy: 0.5912
Epoch 5/5
6/6 [==============================] - 2s 289ms/step - loss: 0.6771 - accuracy: 0.6133


In [78]:
model.evaluate(X_test,y_test)

2/2 [==============================] - 3s 135ms/step - loss: 0.6954 - accuracy: 0.5000


[0.695354700088501, 0.5]